`나는 밥을 []`에서 빈 칸에 들어갈 말이 `먹는다`라는 것을 큰 고민 없이 알 수 있다. `밥`은 통계적으로 `먹`히기 때문이다. `알바생이 커피를 []`라면 아마도 `만든다`가 정답일 것이다.  

인공지능이 글을 이해하게 하는 방식도 위와 같다. 어떤 문법적인 원리를 통해서가 아니고, **수많은 글을 읽게 함으로써** `나는`, `밥을`, 그 다음이 `먹는다`라는 사실을 알게 하는 것이다. 그런 이유에서 **많은 데이터가 곧 좋은 결과**를 만들어낸다.  

이 방식을 가장 잘 처리하는 인공지능 중 하나가 **순환신경망(RNN)**이다.  
앞에서 `먹었다`를 만드는 법은 배웠지만, 가장 첫 시작인 `나는`은 어떻게 만들어야 할까?  

이는 `<start>`라는 특수한 토큰을 맨 앞에 추가해 줌으로써 해결할 수 있다. `<start>`를 입력으로 받은 순환신경망은 다음 단어로 `나는`을 생성하고, **생성한 단어를 다시 입력으로 사용**한다. 이 순환적인 특성을 살려 순환신경망이라고 이름을 붙인 것이다.  

그렇게 순차적으로 `밥을 먹었다.`까지 생성하고 나면, 인공지능은 다 만들었다는 사인으로 `<end>`라는 특수한 토큰을 생성한다. 즉, `<start>`가 문장의 시작에 더해진 입력 데이터(문제지)와, `<end>`가 문장의 끝에 더해진 출력 데이터(답안지)가 필요하며, 이는 **문장 데이터만 있으면 만들어 낼 수 있다**는 것 또한 알 수 있다.

In [1]:
sentence = " 나는 밥을 먹었다 "

source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
print("Target 문장:", target_sentence)

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


#### 언어 모델(Language Model)
`나는`, `밥을`, `먹었다`를 순차적으로 생성할 때, `밥을` 다음이 `먹었다`인 것은 쉽게 알 수 있다. 하지만 `나는` 다음이 `밥을`인 것은 조금 억지처럼 느껴질 수 있다. 실제로 동작하는 방식도, `밥을`을 만드는 것은 순전히 운이다. 의도한다고 해서 나오는 것이 아니다.  

이걸 조금 더 확률적으로 표현해보자.  

'나는 밥을' 다음에 '먹었다'가 나올 확률을 p(먹었다 |  나는, 밥을)이라고 하자. 그렇다면 이 확률은 p(밥을 | 나는)보다는 높게 나올 것이다. 아마 p(먹었다 | 나는, 밥을, 맛있게)의 확률값은 더 높을 것이다.  

어떤 문구 뒤에 다음 단어가 나올 확률이 높다는 것은 그 단어가 나오는 것이 보다 자연스럽다는 뜻이 된다. 그렇다면 '나는' 뒤에 '밥을'이 나오는 것이 자연스럽지 않다는 것일까? 그것은 아니다. '나는' 뒤에 올 수 있는 자연스러운 단어의 경우의 수가 워낙 많기에 불확실성이 높을 뿐이다.  

n-1개의 단어 시퀀스 w1....wn-1가 주어졌을 때, n번째 단어 wn으로 무엇이 올지를 에측하는 확률 모델을 **언어 모델(Language Model)이라고 부른다.  
이런 언어 모델을 어떻게 학습시킬 수 있을까? 간단하다. 어떤 텍스트도 언어 모델의 학습 데이터가 될 수 있다. n-1번째까지의 단어 시퀀스가 x_train이 되고, n번째 단어가 y_train이 되는 데이터셋은 무궁무진하게 만들 수 있기 때문이다.  

이렇게 학습된 언어 모델을 학습 모드가 아닌 테스트 모드로 가동하면 어떤 일이 벌어질까? 이 모델은 일정한 단어 시퀀스가 주어지면 다음 단어, 그 다음 단어를 계속해서 예측해 낼 것이다. 이게 바로 텍스트 생성이고 작문이 된다. **잘 학습된 언어 모델은 훌륭한 문장 생성기**로 동작하게 된다.

In [2]:
import re
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어온다.
file_path = '/content/data/shakespeare.txt'
with open(file_path, "r") as f:
  raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


완벽한 연극 대본이다. 하지만 문장(대사)만을 원하므로 화자 이름이나 공백뿐인 정보는 필요가 없다. 지금 만들 모델은 연극 대사를 만들어 내는 모델이다.  
1차 필터링할 필요가 있으므로 데이터의 형태를 자세히 살피며 필터를 구상해보자.  
우리가 원치 않는 문장은 화자가 표기된 문장, 공백인 문장이다. 화자가 표기된 문장의 끝은 `:`로 끝나는데, 일반적으로 대사가 `:`로 끝나는 일은 없으니, `:`를 기준으로 문장을 제외해도 괜찮을 것이다. 그리고 공백인 문자는 길이를 검사해 길이가 0이라면 제외시킨다.

In [3]:
for idx, sentence in enumerate(raw_corpus):
  if len(sentence) == 0: continue # 길이가 0인 문장은 건너뛴다.
  if sentence[-1] == ":": continue  # 문장의 끝이 :인 문장은 건너뛴다.

  if idx > 9: break # 문장 10개만 확인
  print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


우리가 원하는 문장만 성공적으로 출력된다.  

텍스트 분류 모델과 마찬가지로 텍스트 생성 모델도 단어 사전을 만들게 된다. 그렇다면 문장을 일정한 기준으로 쪼개야하는데, 그 과정을 **토큰화(Tokenize)**라고 한다.  

가장 심플한 방법은 띄어쓰기를 기준으로 나누는 방법인데, 약간의 문제가 있을 수 있다. 

1. Hi, my name is John. ("Hi,", "my", "john."으로 분리됨) - 문장부호
2. First, open the first chapter. (First와 first를 다른 단어로 인식) - 대소문자
3. He is a ten-year-old boy. (ten-year-old를 한 단어로 인식) - 특수문자  

`1`을 막기 위해 **문장 부호 양쪽에 공백을 추가**하고, `2`를 막기 위해 **모든 문자를 소문자료 변환**한다. `3`을 막기 위해 **특수문자는 모두 제거**한다.  

이러한 전처리를 위해 정규표현식(regex)을 이용한 필터링이 유용하게 사용된다.

In [4]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지운다.
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러 개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다.
#     5. 다시 양쪽 공백을 지운다.
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


자연어처리 분야에서 모델의 입력이 되는 문장을 **소스 문장(Source Sentence)**, 정답 역할을 하게 될 모델의 출력 문장을 **타겟 문장(Target Sentence)**라고 관례적으로 부른다. 각각 X_train, y_train에 해당한다고 볼 수 있다.  

위에서 만든 정제 함수를 통해 만든 데이터셋에서 토큰화를 진행한 후, 끝 단어 <end>`를 없애면 소스 문장, 첫 단어 `<start>`를 없애면 타겟 문장이 된다. 

In [5]:
# 여기에 정제된 문장을 모은다.
corpus = []

for sentence in raw_corpus:
  # 원치 않는 문장은 건너뛴다.
  if len(sentence) == 0: continue
  if sentence[-1] == ":": continue

  # 정제를 하고 담는다.
  preprocessed_sentence = preprocess_sentence(sentence)
  corpus.append(preprocessed_sentence)

# 정제된 결과 10개 확인
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

텐서플로우는 자연어 처리를 위한 여러 가지 모듈을 제공하는데, `tf.keras.preprocessing.text.Tokenizer` 패키지는 정제된 데이터를 토큰화하고, 단어 사전(vocabulary / dictionary)을 만들어주며, 데이터를 숫자로 변환까지 한 번에 해준다. 이 과정을 **벡터화(vectorize)**라 하며, 숫자로 변환된 데이터를 **텐서(tensor)**라고 칭한다. 텐서플로우로 만든 모델의 입출력 데이터는 실제로는 모두 이런 텐서로 변환되어 처리되는 것이다.  
>텐서(tensor)는 굉장히 어려운 물리학 및 수학 개념이다. 그 내용을 모두 이해할 필요는 없으나 아래의 웹페이지가 설명하는 간단한 개념 정도는 알고 있으면 좋다.  
 - [Tensor란 무엇인가?](https://rekt77.tistory.com/102)

In [6]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용한다.
# 더 잘 알기 위해 아래 문서들을 참고
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만든다.
    # 이미 문장을 정제했으니 filters가 필요 없다. 
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꾼다.
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성한다.
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환한다.
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춘다.
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춘다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f096aa7b550>


생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력해보자.



In [7]:
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


텐서 데이터는 모두 정수로 이루어져 있다. 이 숫자는 다름 아니라, tokenizer에 구축된 단어 사전의 인덱스다. 단어 사전이 어떻게 구축되었는지 확인해보자.

In [8]:
for idx in tokenizer.index_word:
  print(idx, ":", tokenizer.index_word[idx])

  if idx >=10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


2번 인덱스가 바로 `<start>`였다. 모든 행이 왜 2로 시작하는지 이해할 수 있다.  
이제 생성된 텐서를 소스와 타겟으로 분리해 모델이 학습할 수 있게 하자. 이 과정도 텐서플로우가 제공하는 모듈을 사용할 것이니, 어떻게 사용하는지만 눈여겨 보자.  

텐서 출력부에서 행 뒤에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패딩(padding)을 채워넣은 것이다. 사전에는 없지만 0은 바로 패딩 문자 `<pad>`가 될 것이다.

In [9]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성한다.
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다.
src_input = tensor[:, :-1]
# tensor에서 <start>를 ㅈ랄라내서 타겟 문장을 생성한다.
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


corpus 내의 첫 번째 문장에 대해 생성된 소스와 타겟 문장을 확인해보았다. 예상대로 소스는 2(`<start>`)로 시작해서 3(`<end>`)로 끝난 후 0(`<pad>`)로 채워져 있다. 하지만 타겟은 2로 시작하지 않고 소스를 왼쪽으로 한 칸 시프트 형태를 갖고 있다.  

마지막으로 데이터셋 객체를 생성한다. `tf.data.Dataset` 객체를 생성하는 방법을 흔히 사용한다. `tf.data.Dataset` 객체는 텐서플로우에서 사용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의 기능을 제공한다.  
이미 데이터셋을 텐서 형태로 생성했기 때문에, `tf.data.Dataset.from_tensor_slices()` 메서드를 이용해 `tf.data.Dataset` 객체를 생성한다. 

In [12]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함해 7001개
VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

In [13]:
class TextGenerator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_size, hidden_size):
    super().__init__()

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
    self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
    self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
    self.linear = tf.keras.layers.Dense(vocab_size)

  def call(self, x):
    out = self.embedding(x)
    out = self.rnn_1(out)
    out = self.rnn_2(out)
    out = self.linear(out)

    return out

embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

- `embedding_size` : 워드 벡터의 차원 수, 즉 단어가 추상적으로 표현되는 크기. 
- `hidden_size` : LSTM 레이어의 hidden state의 차원 수

In [14]:
# 데이터셋에서 데이터 한 배치만 로드
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 로드한 데이터를 모델에 넣음
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 3.23315122e-04,  5.84778143e-04, -3.33387434e-04, ...,
          3.11978965e-06,  8.72639357e-05,  1.73452878e-04],
        [ 3.75213276e-04,  8.24246963e-04, -2.05786942e-04, ...,
          3.16691112e-05,  1.23073536e-04,  4.08579537e-04],
        [ 4.12694673e-04,  1.09086931e-03,  7.20844400e-05, ...,
          8.27757540e-05, -5.55248407e-05,  3.82970000e-04],
        ...,
        [-9.54871590e-04,  1.37437345e-03,  4.91933338e-03, ...,
          1.49647368e-03,  5.09794382e-03, -1.11690475e-04],
        [-8.77874554e-04,  1.44473708e-03,  5.46943303e-03, ...,
          1.60472211e-03,  5.59673365e-03, -1.61095479e-04],
        [-7.87832600e-04,  1.51223596e-03,  5.96571201e-03, ...,
          1.70027965e-03,  6.01692731e-03, -1.93378888e-04]],

       [[ 3.23315122e-04,  5.84778143e-04, -3.33387434e-04, ...,
          3.11978965e-06,  8.72639357e-05,  1.73452878e-04],
        [ 2.20956863e-05,  1.10835792e-03, -6.

모델의 최종 출력 텐서 shape은 `shape=(256, 20, 7001)`이다. 7001은 Dense ㄹ이어의 출력 차원 수, 256은 배치 사이즈, 20은 시퀀스 길이다. 

In [15]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1792256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  7176025   
                                                                 
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [16]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 37s 365ms/step - loss: 3.5592
Epoch 2/30
93/93 [==============================] - 34s 364ms/step - loss: 2.8219
Epoch 3/30
93/93 [==============================] - 34s 364ms/step - loss: 2.7440
Epoch 4/30
93/93 [==============================] - 34s 365ms/step - loss: 2.6550
Epoch 5/30
93/93 [==============================] - 34s 364ms/step - loss: 2.5797
Epoch 6/30
93/93 [==============================] - 34s 364ms/step - loss: 2.5260
Epoch 7/30
93/93 [==============================] - 34s 362ms/step - loss: 2.4824
Epoch 8/30
93/93 [==============================] - 34s 361ms/step - loss: 2.4335
Epoch 9/30
93/93 [==============================] - 34s 361ms/step - loss: 2.3895
Epoch 10/30
93/93 [==============================] - 34s 363ms/step - loss: 2.3511
Epoch 11/30
93/93 [==============================] - 34s 362ms/step - loss: 2.3137
Epoch 12/30
93/93 [==============================] - 34s 360ms/step - loss: 2.2777
Epoch 13/30
9

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
  # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
  test_input = tokenizer.texts_to_sequences([init_sentence])
  test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
  end_token = tokenizer.word_index["<end>"]

  while True:
    predict = model(test_tensor)
    predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
    test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
    
    if predict_word.numpy()[0] == end_token: break
    if test_tensor.shape[1] >= max_len: break

  generated = ""
  for word_index in test_tensor[0].numpy():
    generated += tokenizer.index_word[word_index] + " "

  return generated


In [23]:
generate_text(model, tokenizer, init_sentence="<start> i")

'<start> i ll tell you , sir , i ll not be slack to do . <end> '